
# Hello world

In this unit you will learn how to use Python to implement the first ever program
that *every* programmer starts with. This also serves as an example for the master
notebook format.

```
# All markdown cells are searched for triple-backtick blocks. Within a triple quoted block,
# a match on /^# ASSIGNMENT METADATA/ will trigger handling this as an assignment-level metadata
# block. The rest of the metadata is parsed as YAML and may be used e.g. for setting
# default settings for autograder isolation (memory limit etc).
# The assignment_id is useful to identify which assignment a submission pertains to.
# ASSIGNMENT METADATA
assignment_id: "HelloWorld"
```

## Introduction

Here is the traditional first programming exercise, called "Hello world".
The task is to print the message: "Hello, world".

Here are a few examples to get you started. Run the following cells and see how
you can print a message. To run a cell, click with mouse inside a cell, then
press Ctrl+Enter to execute it. If you want to execute a few cells sequentially,
then press Shift+Enter instead, and the focus will be automatically moved
to the next cell as soon as one cell finishes execution.

In [56]:
print("hello")

hello


In [57]:
print("bye bye")

bye bye


In [58]:
print("hey", "you")

hey you


In [59]:
print("one")
print("two")

one
two


## Exercise 1: printing greeting

```
# The markdown cell with triple-backtick block matching /^# EXERCISE METADATA/ is an exercise-level
# metadata. The next block is assumed to be the solution block, and will get annotated with
# the exercise_id.
# EXERCISE METADATA
exercise_id: "hello1"
```

Now it is your turn. Please create a program in the next cell that would print a message "Hello, world":

In [60]:
""" # BEGIN PROMPT
   # ... put your program here
   pass
""" # END PROMPT
def printHello():
    # BEGIN SOLUTION
    print("Hello, world")
    # END SOLUTION

In [61]:
# This will not be included in the student notebook because of BEGIN UNITTEST marker below.

# Imitate the 'import submission' environment with ad-hoc object.
from types import SimpleNamespace
submission = SimpleNamespace(printHello=printHello)

# BEGIN UNITTEST
import unittest

import sys
import io
from contextlib import contextmanager
from io import StringIO

@contextmanager
def capture_output():
    capture_out, capture_err = StringIO(), StringIO()
    save_out, save_err = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = capture_out, capture_err
        yield sys.stdout, sys.stderr
    finally:
        sys.stdout, sys.stderr = save_out, save_err

class HelloOutputTest(unittest.TestCase):
    def test_output(self):
        with capture_output() as (out, err):
            submission.printHello()
        self.assertEqual(err.getvalue(), "")
        self.assertEqual(out.getvalue(), "Hello, world\n")

# END UNITTEST

# The parts after END UNITTEST are executed in the notebook environment, but not copied
# to the autograder scripts or to student notebooks.

# TODO(salikh): Move this into a shared library and make that library installable via pip.
class SummaryTestResult(unittest.TextTestResult):
    """A small extension of TextTestResult that also collects a map of test statuses.
    
    
    result.results is a map from test name (string) to boolean: True(passed) or False(failed or error)"""
    
    separator1 = '=' * 70
    separator2 = '-' * 70
    
    def __init__(self, stream, descriptions, verbosity):
        super(unittest.TextTestResult, self).__init__(stream, descriptions, verbosity)
        # A map of test name to True(passed) or False(failed or error)
        self.results = {}
        # Copied from TextTestResult.
        self.stream = stream
        self.showAll = verbosity > 1
        self.dots = verbosity == 1
        self.descriptions = descriptions

    def testName(self, test):
        """A helper function to format the test as a human-readable string.
        
        The format is TestClassName.test_method. This is similar
        to TextTestResult.getDescription(test), but uses different format.
        getDescription: 'test_one (__main__.HelloTest)'
        testName: 'HelloTest.test_one'
        """
        return unittest.util.strclass(test.__class__).replace('__main__.', '') + '.' + test._testMethodName
        
    def getDescription(self, test):
        doc_first_line = test.shortDescription()
        if self.descriptions and doc_first_line:
            return '\n'.join((str(test), doc_first_line))
        else:
            return str(test)

    def startTest(self, test):
        super(unittest.TextTestResult, self).startTest(test)
        if self.showAll:
            self.stream.write(self.getDescription(test))
            self.stream.write(" ... ")
            self.stream.flush()

    def addSuccess(self, test):
        super(unittest.TextTestResult, self).addSuccess(test)
        if self.showAll:
            self.stream.writeln("ok")
        elif self.dots:
            self.stream.write('.')
            self.stream.flush()
        self.results[self.testName(test)] = True

    def addError(self, test, err):
        super(unittest.TextTestResult, self).addError(test, err)
        if self.showAll:
            self.stream.writeln("ERROR")
        elif self.dots:
            self.stream.write('E')
            self.stream.flush()
        self.results[self.testName(test)] = False

    def addFailure(self, test, err):
        super(unittest.TextTestResult, self).addFailure(test, err)
        if self.showAll:
            self.stream.writeln("FAIL")
        elif self.dots:
            self.stream.write('F')
            self.stream.flush()
        self.results[self.testName(test)] = False

    def addSkip(self, test, reason):
        super(unittest.TextTestResult, self).addSkip(test, reason)
        if self.showAll:
            self.stream.writeln("skipped {0!r}".format(reason))
        elif self.dots:
            self.stream.write("s")
            self.stream.flush()

    def addExpectedFailure(self, test, err):
        super(unittest.TextTestResult, self).addExpectedFailure(test, err)
        if self.showAll:
            self.stream.writeln("expected failure")
        elif self.dots:
            self.stream.write("x")
            self.stream.flush()

    def addUnexpectedSuccess(self, test):
        super(unittest.TextTestResult, self).addUnexpectedSuccess(test)
        if self.showAll:
            self.stream.writeln("unexpected success")
        elif self.dots:
            self.stream.write("u")
            self.stream.flush()

    def printErrors(self):
        if self.dots or self.showAll:
            self.stream.writeln()
        self.printErrorList('ERROR', self.errors)
        self.printErrorList('FAIL', self.failures)

    def printErrorList(self, flavour, errors):
        for test, err in errors:
            self.stream.writeln(self.separator1)
            self.stream.writeln("%s: %s" % (flavour,self.getDescription(test)))
            self.stream.writeln(self.separator2)
            self.stream.writeln("%s" % err)

import sys
import io
suite = unittest.TestLoader().loadTestsFromTestCase(HelloOutputTest)
errors = io.StringIO()
result = unittest.TextTestRunner(verbosity=4,stream=errors, resultclass=SummaryTestResult).run(suite)
# Optional.
print(errors.getvalue())

print(result.results)

test_output (__main__.HelloOutputTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK

{'HelloOutputTest.test_output': True}


## Exercise 2: returning greeting as value

```
# EXERCISE METADATA
exercise_id: "hello2"
```

Please create a function that given a name string returns a string with a greeting,
For example, for input `"world"` it should return `"Hello, world"`.

In [62]:
def hello(name):
    """ # BEGIN PROMPT
    # Please put your solution here:
    # return ...
    pass
    """ # END PROMPT
    # BEGIN SOLUTION
    return "Hello, " + name
    # END SOLUTION

In [63]:
# TEST
assert(hello("world") == "Hello, world")

A code cell marked with `"# TEST"` will be converted into a unit test for the solution. It will also be preserved in the student version of the notebook.

In [64]:
# The part before "BEGIN UNITTEST" -- preamble -- sets up the environment so that 'submission.hello'
# is a function that we need to test. In the autograder worker environment, the preamble will be
# replaced with 'import submission' with an assumption that the student's solution will be written
# to the file 'submission.py'.
submission = SimpleNamespace(hello=hello)

# BEGIN UNITTEST
# The unit tests main part is contained between "BEGIN UNITTEST" and "END UNITTEST". It will be copied
# verbatim into the autograder directory, with an addition of 'import submission'

import unittest

class HelloTest(unittest.TestCase):
    def test_one(self):
        self.assertEqual(submission.hello("one"), "Hello, one")
        
    def test_bad(self):
        self.assertEqual(submission.hello("bad"), "Hello, good")
        
# END UNITTEST

import sys
import io
suite = unittest.TestLoader().loadTestsFromTestCase(HelloTest)
errors = io.StringIO()
result = unittest.TextTestRunner(verbosity=4,stream=errors, resultclass=SummaryTestResult).run(suite)
# Optional.
print(errors.getvalue())

print(result.results)

test_bad (__main__.HelloTest) ... FAIL
test_one (__main__.HelloTest) ... ok

FAIL: test_bad (__main__.HelloTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-64-6dbf578cd0e2>", line 18, in test_bad
    self.assertEqual(submission.hello("bad"), "Hello, good")
AssertionError: 'Hello, bad' != 'Hello, good'
- Hello, bad
?        ^^
+ Hello, good
?        ^^^


----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=1)

{'HelloTest.test_bad': False, 'HelloTest.test_one': True}


In [65]:
# BEGIN AUTOTEST
# The tests below test that the unit test suite above produces expected outcomes from
# TODO(salikh): Figure out the right syntax for autotests.
def wrong_hello(name):
    return "Bye, " + name

import unittest

submission.hello = wrong_hello
suite = unittest.TestLoader().loadTestsFromTestCase(HelloTest)
errors = io.StringIO()
result = unittest.TextTestRunner(verbosity=4,stream=errors,  resultclass=SummaryTestResult).run(suite)

print(result.results)
assert(result.results['HelloTest.test_bad'] == False)
assert(result.results['HelloTest.test_one'] == False)

{'HelloTest.test_bad': False, 'HelloTest.test_one': False}


In [66]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_cell_magic
def submission(line, cell):
    """A magic to mark a textual submission, possibly incorrect and unparseable."""
    return cell

@register_line_magic
def autotest(line):
    """A magic to run an auto test: run previously defined unit tests with a submission just given."""
    return eval(line)

del submission, autotest

In [67]:
%%submission
def syntax_error(name:
    return name

'def syntax_error(name:\n    return name\n'

In [68]:
result = %autotest HelloTest
result
#assert(result['HelloTest.test_bad'] == False)

__main__.HelloTest

In [69]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def lmagic(line):
    "my line magic"
    return line

@register_cell_magic
def cmagic(line, cell):
    "my cell magic"
    return cell

del lmagic, cmagic

In [70]:
x = %lmagic a b c
print("Hello, " +x)

Hello, a b c


In [71]:
%%cmagic
my cell magic
asfd

'my cell magic\nasfd\n'

In [72]:
vars()

{'__name__': '__main__',
 '__doc__': ' # BEGIN PROMPT\n   # ... put your program here\n   pass\n',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'print("hello")',
  'print("bye bye")',
  'print("hey", "you")',
  'print("one")\nprint("two")',
  '""" # BEGIN PROMPT\n   # ... put your program here\n   pass\n""" # END PROMPT\ndef printHello():\n    # BEGIN SOLUTION\n    print("Hello, world")\n    # END SOLUTION',
  '# This will not be included in the student notebook because of BEGIN UNITTEST marker below.\n\n# Imitate the \'import submission\' environment with ad-hoc object.\nfrom types import SimpleNamespace\nsubmission = SimpleNamespace(printHello=printHello)\n\n# BEGIN UNITTEST\nimport unittest\n\nimport sys\nimport io\nfrom contextlib import contextmanager\nfrom io import StringIO\n\n@contextmanager\ndef capture_output():\n    capture_out, capture_err = Strin

In [73]:
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MyMagics(Magics):

    @line_magic
    def lmagic(self, line):
        "my line magic"
        print("Full access to the main IPython object:", self.shell)
        print("Variables in the user namespace:", list(self.shell.user_ns.keys()))
        return line

    @cell_magic
    def cmagic(self, line, cell):
        "my cell magic"
        return line, cell

    @line_cell_magic
    def lcmagic(self, line, cell=None):
        "Magic that works both as %lcmagic and as %%lcmagic"
        if cell is None:
            print("Called as line magic")
            return line
        else:
            print("Called as cell magic")
            return line, cell
        
ip = get_ipython()
ip.register_magics(MyMagics)

In [74]:
%lmagic ab cd ef

Full access to the main IPython object: <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fc6f8cd84e0>
Variables in the user namespace: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', '_i3', '_i4', '_i5', 'printHello', '_i6', 'SimpleNamespace', 'unittest', 'sys', 'io', 'contextmanager', 'StringIO', 'capture_output', 'HelloOutputTest', 'SummaryTestResult', 'suite', 'errors', 'result', '_i7', 'hello', '_i8', '_i9', 'HelloTest', '_i10', 'wrong_hello', '_i11', '_i12', 'register_line_magic', 'register_cell_magic', 'register_line_cell_magic', '_i13', '_i14', '_i15', '_15', '_i16', '_i17', '_i18', '_i19', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_28', '_i29', '_i30', '_i31', '_31', '_i32', '_i33', '_33', '_i34', '_i35', '_35', '_i36', '_36', '_i37', '_i38', '_i39', '_i40', '_i41', '_i42', '_i43'

'ab cd ef'